<a href="https://colab.research.google.com/github/Siddhrajs/LeavesDataset/blob/main/LeavesDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Drive Initialization
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#all imports and region growing function
#Superpixel Segmentation
import scipy.ndimage as ndimage
from skimage import color
import matplotlib.pyplot as plt
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage import io
import matplotlib.pyplot as plt
import argparse
import numpy as np
import cv2

#region growing function

def white_region(img,seed):
  new = set()
  que = set()
  que.add((seed[0],seed[1]))
  
  neighbourx = np.array([0,0,1,-1])
  neighboury = np.array([1,-1,0,0])
  
  while que:
    tx,ty = que.pop()
    for i in range(4):
      if tx+neighbourx[i]>=0 and tx+neighbourx[i]<len(lab) and ty+neighboury[i]>=0 and ty+neighboury[i]<len(lab[0]) and img[tx+neighbourx[i],ty+neighboury[i]]==0 and (tx+neighbourx[i], ty+neighboury[i]) not in new:
        new.add((tx+neighbourx[i],ty+neighboury[i]))
        que.add((tx+neighbourx[i],ty+neighboury[i]))
  
  del que
  del neighbourx
  del neighboury
  finseg = np.zeros(shape=np.shape(img))
  #print(new)
  for a,b in new:
    finseg[a,b] = 255
    
  del new
    
  #plt.imshow(finseg)
  #plt.show()
  
  return finseg

In [ ]:
for iteri in range(7):
  #superpixel segmentation
  print('start')
  print(iteri)
  print(1)
  img = ndimage.imread('/content/drive/My Drive/images/eggplant'+str(iteri+1)+'.jpg')

  lab = color.rgb2lab(img)

  image = img_as_float(img)

  numSegments = 800
  segments = slic(image, n_segments = numSegments, sigma = 5)
  del image
  #plt.imshow(img)
  #plt.show()
  
  #foreground extraction
  print(2)
  
  temp = [[[np.uint8(255) for z in range(3)] for y in range(len(lab[0]))] for x in range(len(lab))]
  count = [0 for x in range(numSegments*2)]
  #print(len(temp[0]))
  limit = len(lab[0])*len(lab)/numSegments
  del numSegments
  limit = limit/2
  for i in range(len(lab)):
    for j in range(len(lab[0])):
      if lab[i][j][1]<-5:
        count[segments[i][j]]=count[segments[i][j]]+1
  #print(count)
  for i in range(len(lab)):
    for j in range(len(lab[0])):
      if count[segments[i][j]]>limit:
        temp[i][j][0]=img[i][j][0]
        temp[i][j][1]=img[i][j][1]
        temp[i][j][2]=img[i][j][2]
        
  del count
  del limit
        
  for iterj in range(5):
    #Distance Map Calculation
    print('substart')
    print(iterj)
    print(11)

    distMap = [[max(len(lab[0]),len(lab)) for y in range(len(lab[0]))] for x in range(len(lab))]
    distMapColor = [[[np.uint8(255) for z in range(3)] for y in range(len(lab[0]))] for x in range(len(lab))]
    #print(distMap[0][0])
    for i in range(len(lab)):
      for j in range(len(lab[i])):
        if i==0 or j==0 or i==len(lab)-1 or j==len(lab[i])-1:
          distMap[i][j]=1

    for i in range(len(lab)):
      for j in range(len(lab[i])):
        if temp[i][j][0]==255 and temp[i][j][1]==255 and temp[i][j][2]==255:
          distMap[i][j]=0

    for j in range(len(lab[0])):
      prev=0
      for i in range(len(lab)):
        distMap[i][j]=min(distMap[i][j],prev+1)
        prev=distMap[i][j]
    
    for j in range(len(lab[0])):
      prev=0
      for i in range(len(lab)):
        distMap[len(lab)-i-1][j]=min(distMap[len(lab)-i-1][j],prev+1)
        prev=distMap[len(lab)-i-1][j]
    
    for i in range(len(lab)):
      prev=0
      for j in range(len(lab[i])):
        distMap[i][j]=min(distMap[i][j],prev+1)
        prev=distMap[i][j]

    for i in range(len(lab)):
      prev=0
      for j in range(len(lab[i])):
        distMap[i][len(lab[i])-j-1]=min(distMap[i][len(lab[i])-j-1],prev+1)
        prev=distMap[i][len(lab[i])-j-1]
    
    Max = 0

    for i in range(len(lab)):
      for j in range(len(lab[0])):
        Max = max(Max,distMap[i][j])
    
    for i in range(len(lab)):
      for j in range(len(lab[0])):
        distMapColor[i][j][0]=0
        distMapColor[i][j][1]=0
    
    for i in range(len(lab)):
      for j in range(len(lab[0])):
        distMapColor[i][j][0]=distMap[i][j]/Max
        distMapColor[i][j][2]=1-distMap[i][j]/Max

    #plt.imshow(temp, interpolation='nearest')
    #plt.show()
    #plt.imshow(distMapColor, interpolation='nearest')
    #plt.show()
    del distMapColor

    mainSeg = 0
    for i in range(len(lab)):
      for j in range(len(lab[0])):
        if distMap[i][j]==Max:
          mainSeg=segments[i][j]
    del mainSeg
    
    #smoothening leaves
    print(12)

    rgImage = cv2.cvtColor(np.asarray(temp), cv2.COLOR_BGR2GRAY)

    #plt.imshow(rgImage)
    #plt.show()
    del rgImage

    dst = cv2.fastNlMeansDenoisingColored(np.asarray(temp),None, 10, 10, 117, 15)

    #plt.imshow(cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY))
    #plt.show()

    #dst2 = cv2.fastNlMeansDenoisingColored(dst, None, 10, 10, 7, 15)

    #plt.imshow(cv2.cvtColor(dst2, cv2.COLOR_BGR2GRAY))
    #plt.show()

    #dst3 = cv2.fastNlMeansDenoisingColored(dst2, None, 10, 10, 7, 15)

    #plt.imshow(cv2.cvtColor(dst3, cv2.COLOR_BGR2GRAY))
    #plt.show()

    kernel = np.ones((5,5),np.float32)/25
    dst4 = cv2.filter2D(dst,-1,kernel)
    del dst

    #plt.imshow(cv2.cvtColor(dst4, cv2.COLOR_BGR2GRAY))
    #plt.show()
    
    #edge detection
    print(13)

    #plt.imshow(dst4, interpolation='nearest')
    #plt.show()
    blur = cv2.GaussianBlur(dst4, (51, 51), 0) 
    del dst4
    #plt.imshow(blur, interpolation='nearest')
    #plt.show()
    edges = cv2.Canny(blur, 0,10)
    del blur
    #plt.imshow(edges[0:500,500:1000], interpolation='nearest')
    #plt.show()

    #krnl = np.ones((15,15),np.float32)/(15*15)
    edgesHard = cv2.filter2D(edges,-1,kernel)
    del edges

    for i in range(len(lab)):
      for j in range(len(lab[0])):
        if edgesHard[i,j]!=0:
          edgesHard[i,j]=255

    #plt.imshow(edgesHard)
    #plt.show()

    edgesDHard = cv2.filter2D(edgesHard,-1,kernel)
    del edgesHard

    for i in range(len(lab)):
      for j in range(len(lab[0])):
        if edgesDHard[i,j]!=0:
          edgesDHard[i,j]=255

    #plt.imshow(edgesDHard)
    #plt.show()

    edgesQHard = cv2.filter2D(edgesDHard,-1,kernel)
    del edgesDHard

    for i in range(len(lab)):
      for j in range(len(lab[0])):
        if edgesQHard[i,j]!=0:
          edgesQHard[i,j]=255

    #plt.imshow(edgesQHard)
    #plt.show()

    edgesOHard = cv2.filter2D(edgesQHard,-1,kernel)
    del kernel
    del edgesQHard

    for i in range(len(lab)):
      for j in range(len(lab[0])):
        if edgesOHard[i,j]!=0:
          edgesOHard[i,j]=255

    #plt.imshow(edgesOHard)
    #plt.show()
    
    #leaf mask
    print(14)

    seed = [0 for x in range(2)]

    for i in range(len(lab)):
      for j in range(len(lab[0])):
        if distMap[i][j]==Max:
          seed[0]=i
          seed[1]=j

    #print(seed)
    #print(edgesOHard[seed[0],seed[1]])
    del distMap
    del Max

    finseg = white_region(edgesOHard,seed)
    del seed
    del edgesOHard

    gray = finseg.astype(np.uint8)
    #print(type(finseg[0,0]))
    ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    del ret
    
    #plt.imshow(thresh)
    #plt.show()
    
    #leaf mask correction
    print(15)

    krnl = np.ones((171,171),np.float32)/(171*171)
    rmstem = cv2.filter2D(finseg,-1,krnl)
    del krnl

    #plt.imshow(rmstem)
    #plt.show()

    for i in range(len(lab)):
      for j in range(len(lab[0])):
        if rmstem[i,j]>90:
          rmstem[i,j]=255

    #plt.imshow(rmstem)
    #plt.show()

    gray = rmstem.astype(np.uint8)
    #print(type(finseg[0,0]))
    ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    del ret

    for i in range(len(lab)):
      for j in range(len(lab[0])):
        if finseg[i,j]==255:
          rmstem[i,j]=255

    #plt.imshow(rmstem)
    #plt.show()

    gray = rmstem.astype(np.uint8)
    del finseg
    #print(type(finseg[0,0]))
    ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    
    del ret
    del gray
    del rmstem
    
    #extract leaf
    print(16)

    tmp = [[[np.uint8(255) for z in range(3)] for y in range(len(lab[0]))] for x in range(len(lab))]
    leaf = [[[np.uint8(255) for z in range(3)] for y in range(len(lab[0]))] for x in range(len(lab))]

    for i in range(len(lab)):
      for j in range(len(lab[0])):
        if thresh[i,j]!=0:
          tmp[i][j][0]=temp[i][j][0]
          tmp[i][j][1]=temp[i][j][1]
          tmp[i][j][2]=temp[i][j][2]
        else:
          leaf[i][j][0]=img[i][j][0]
          leaf[i][j][1]=img[i][j][1]
          leaf[i][j][2]=img[i][j][2]

    del temp
    temp = tmp
    del thresh

    #plt.imshow(temp, interpolation='nearest')
    #plt.show()
    #plt.imshow(leaf, interpolation='nearest')
    #plt.show()
    
    #store leaf as image
    print(17)
    cv2.imwrite('/content/drive/My Drive/images/leaves/leaf'+str(5*iteri+iterj+1)+'.jpg',np.array(leaf))
    del leaf
    print('finish')
  del lab
  del img
  del segments

start
0
1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  


2
substart
0
11
12
13
14
15
16
17
finish
substart
1
11
12
13
14
15
16
17
finish
substart
2
11
12
13
14
15
16
17
finish
substart
3
11
12
13
14
15
16
17
finish
substart
4
11
12
13
14
15
16
17
finish
start
1
1
2
substart
0
11
12
13
14
15
16
17
finish
substart
1
11
12
13
14
15
16
17
finish
substart
2
11
12
13
14
15
16
17
finish
substart
3
11
12
13
14
15
16
17
finish
substart
4
11
12
13
14
15
16
17
finish
start
2
1
2
substart
0
11
12
13
14
15
16
17
finish
substart
1
11
12
13
14
15
16
17
finish
substart
2
11
12
13
14
15
16
17
finish
substart
3
11
12
13
14
15
16
17
finish
substart
4
11
12
13
14
15
16
17
finish
start
3
1
2
substart
0
11
12
13
14
15
16
17
finish
substart
1
11
12
13
14
15
16
17
finish
substart
2
11
12
13
14
15
16
17
finish
substart
3
11
12
13
14
15
16
17
finish
substart
4
11
12
13
14
15
16
17
finish
start
4
1
2
substart
0
11
12
13
14
15
16
17
finish
substart
1
11
12
13
14
15
16
17
finish
substart
2
11
12
13
14
15
16
17
finish
substart
3
11
12
13
14
15
16
17
finish
substart
4
11
